<a href="https://colab.research.google.com/github/danshal/DL_project/blob/develop/wav2vec_SV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install soundfile
!pip install fairseq


     |████████████████████████████████| 1.7MB 5.3MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 133kB 21.8MB/s 
     |████████████████████████████████| 112kB 19.0MB/s 
     |████████████████████████████████| 645kB 18.7MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141231 sha256=0f39136313fd14764caeda644b968a6ebf0760c431716d9a9ed38f7494f453d0
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#!/usr/bin/env python3
# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

"""
Helper script to pre-compute embeddings for a flashlight (previously called wav2letter++) dataset
"""

import argparse
import glob
import os
import os.path as osp
import pprint

import soundfile as sf
import torch
import fairseq
from torch import nn
from torch.utils.data import DataLoader


try:
    import tqdm
except:
    print("Install tqdm to use --log-format=tqdm")

In [ ]:

cp_path = '/content/drive/My Drive/DeepProject/wav2vec/wav2vec_small.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

wav_input_16khz = torch.randn(1,10000)
z = model.feature_extractor(wav_input_16khz)

In [83]:
print(wav_input_16khz.shape)
print(z.shape)

torch.Size([1, 10000])
torch.Size([1, 512, 31])
tensor([[ 0.7477,  0.9358,  1.0242,  ..., -0.1308, -1.2485, -1.3395]])


##Import 2 examples from dataset and check the similarity between two same class examples

In [19]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

--2021-01-23 17:45:57--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M  25.0MB/s    in 15s     

2021-01-23 17:46:12 (21.2 MB/s) - ‘dev-clean.tar.gz’ saved [337926286/337926286]



In [24]:
!tar -xf dev-clean.tar.gz

In [109]:
def read_audio(fname):
    """ Load an audio file and return PCM along with the sample rate """

    wav, sr = sf.read(fname)
    assert sr == 16e3

    return wav, 16e3


def audio2tensor(fname):
  input = read_audio(fname)
  input_tensor = torch.from_numpy(input).float()
  input_tensor = input_tensor.resize(1, input1_tensor.size(0))

wav2vec_utils = '/fairseq/'
speaker_84_data_dir = 'LibriSpeech/dev-clean/84'
input1, _ = read_audio(f'{speaker_84_data_dir}/121123/84-121123-0003.flac')
input2, _ = read_audio(f'{speaker_84_data_dir}/121123/84-121123-0004.flac')
input1_tensor = torch.from_numpy(input1).float()
input2_tensor = torch.from_numpy(input2).float()
print(f'input1 size is = {input1_tensor.shape} ; input2 size is = {input2_tensor.shape}')
input1_tensor = input1_tensor.resize(1, input1_tensor.size(0))
input2_tensor = input2_tensor.resize(1, input2_tensor.size(0))
#Pass them through the encoder
output1 = model.feature_extractor(input1_tensor)
output2 = model.feature_extractor(input2_tensor)
print(output1.shape)
print(output2.shape)

input1 size is = torch.Size([108800]) ; input2 size is = torch.Size([70400])


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


torch.Size([1, 512, 339])
torch.Size([1, 512, 219])


##This is the feature extractor of the model

In [112]:
class ConvFeatureExtractionModel(nn.Module):
    def __init__(
        self,
        conv_layers,
        dropout,
        log_compression,
        skip_connections,
        residual_scale,
        non_affine_group_norm,
        activation,
    ):
        super().__init__()

        def block(n_in, n_out, k, stride):
            return nn.Sequential(
                nn.Conv1d(n_in, n_out, k, stride=stride, bias=False),
                nn.Dropout(p=dropout),
                norm_block(
                    is_layer_norm=False, dim=n_out, affine=not non_affine_group_norm
                ),
                activation,
            )

        in_d = 1
        self.conv_layers = nn.ModuleList()
        for dim, k, stride in conv_layers:
            self.conv_layers.append(block(in_d, dim, k, stride))
            in_d = dim

        self.log_compression = log_compression
        self.skip_connections = skip_connections
        self.residual_scale = math.sqrt(residual_scale)

    def forward(self, x):
        # BxT -> BxCxT
        x = x.unsqueeze(1)

        for conv in self.conv_layers:
            residual = x
            x = conv(x)
            if self.skip_connections and x.size(1) == residual.size(1):
                tsz = x.size(2)
                r_tsz = residual.size(2)
                residual = residual[..., :: r_tsz // tsz][..., :tsz]
                x = (x + residual) * self.residual_scale

        if self.log_compression:
            x = x.abs()
            x = x + 1
            x = x.log()

        return x
  